<a href="https://colab.research.google.com/github/likw99/awesome_colab/blob/main/IOPaint_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title IOPaint in Google Colab
#@markdown In order to use this colab:
#@markdown - Enter your [ngrok auth token](https://dashboard.ngrok.com/get-started/your-authtoken)
#@markdown - Select a model ([find documentation here](https://www.iopaint.com/models))
#@markdown - Select the T4 GPU accelerator at `Runtime > Change runtime type > Hardware accelerator`
#@markdown - Click `Runtime > Run all`
#@markdown - Follow the link to access the UI

#@markdown Note: beware that installing models for the first time will take some time.
ngrok_token = "27wkTJk6QE901Le8D1b2xQOMzOX_3UBxQxEb3n5yBNi4bs7CQ" # @param {type:"string"}
model = "lama" # @param ["lama", "mat", "migan", "ldm", "zits", "fcf", "manga", "Sanster/PowerPaint-V1-stable-diffusion-inpainting", "Sanster/AnyText", "timbrooks/instruct-pix2pix", "Fantasy-Studio/Paint-by-Example", "kandinsky-community/kandinsky-2-2-decoder-inpaint"]

# Download and install ollama to the system
!pip install iopaint pyngrok > install_logs.txt

import os
import asyncio
from pyngrok import ngrok
import sys

os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

print("Remember, installing models for the first time will be very slow! Be patient.")

async def run_process(cmd):
  print('>>> starting', *cmd)
  p = await asyncio.subprocess.create_subprocess_exec(
      *cmd,
      stdout=asyncio.subprocess.PIPE,
      stderr=asyncio.subprocess.PIPE,
  )

  async def pipe(lines):
    async for line in lines:
      print(line.strip().decode('utf-8'))

  await asyncio.gather(
      pipe(p.stdout),
      pipe(p.stderr),
  )

if not ngrok_token:
  print("Hey! You haven't set the Ngrok token. Without the token, the app won't start. Please paste your token in the input.")
  sys.exit("Ngrok token not set")

ngrok.set_auth_token(ngrok_token)
active_tunnels = ngrok.get_tunnels()
for tunnel in active_tunnels:
    public_url = tunnel.public_url
    ngrok.disconnect(public_url)
url = ngrok.connect(addr="8000", bind_tls=True)
print(f"IOPaint public URL: {url}")
print("Please wait until the \"Application startup complete.\" message appears...")

await asyncio.gather(
    run_process(['iopaint', 'start', '--model', model, '--device', 'cuda', '--port', '8000']),
)

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.
Remember, installing models for the first time will be very slow! Be patient.
IOPaint public URL: NgrokTunnel: "https://1f22-34-125-27-170.ngrok-free.app" -> "http://localhost:8000"
Please wait until the "Application startup complete." message appears...
>>> starting iopaint start --model lama --device cuda --port 8000
2024-02-06 05:45:58.899 | INFO     | iopaint.runtime:setup_model_dir:82 - Model directory: /root/.cache
- Platform: Linux-6.1.58+-x86_64-with-glibc2.35
- Pytho